# 2.4節
データ主導プログラミング

In [1]:
(define (square x) (* x x))
(define (cube x) (* x x x))
(define pi 3.141592)
(define (show-lines . s)
  (cond ((null? s) 'shown)
        (else (display (car s))
              (newline)
              (apply show-lines (cdr s)))))

テーブル操作用（拾い物）

In [2]:
(define global-array '())

(define (make-entry k v) (list k v))
(define (key entry) (car entry))
(define (value entry) (cadr entry))

(define (put op type item)
  (define (put-helper k array)
    (cond ((null? array) (list(make-entry k item)))
          ((equal? (key (car array)) k) array)
          (else (cons (car array) (put-helper k (cdr array))))))
  (set! global-array (put-helper (list op type) global-array)))

(define (get op type)
  (define (get-helper k array)
    (cond ((null? array) #f)
          ((equal? (key (car array)) k) (value (car array)))
          (else (get-helper k (cdr array)))))
  (get-helper (list op type) global-array))

タグ付きデータの扱いと、タグによる手続きの使い分け

In [3]:
(define (attach-tag type-tag contents)
  (cons type-tag contents))
(define (type-tag datum)
  (if (pair? datum) (car datum)
      (error "Bad tagged datum: TYPE-TAG" datum)))
(define (contents datum)
  (if (pair? datum) (cdr datum)
      (error "Bad tagged datum: CONTENTS" datum)))

(define (apply-generic op . args)
  (let* ((type-tags (map type-tag args))
         (proc (get op type-tags)))
    (if proc
        (apply proc (map contents args))
        (error "No method for those types: APPLY-GENERIC" (list op type-tags)))))

## 複素数計算

直交座標系表現に基づいた複素数パッケージ

In [4]:
(define (install-rectangular-package)
  ;;internal procedure
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y) (cons x y))
  (define (magnitude z)
    (sqrt (+ (square (real-part z)) (square (imag-part z)))))
  (define (angle z)
    (atan (imag-part z) (real-part z)))
  (define (make-from-mag-angle r a)
    (cons (* r (cos a)) (* r (sin a))))
  
  ;;interface to the rest of the system
  (define (tag x) (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular
       (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular
       (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

極座標表示に基づいた複素数パッケージ

In [5]:
(define (install-polar-package)
  ;;internal procedure
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang x y) (cons x y))
  (define (real-part z) (* (magnitude z) (cos (angle z))))
  (define (imag-part z) (* (magnitude z) (sin (angle z))))
  (define (make-from-real-imag x y)
    (cons (sqrt (+ (square x) (square y))) (atan y x)))
  
  ;;interface to the rest of the system
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar
       (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar
       (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

パッケージのインストールとジェネリック手続きの定義

In [6]:
(install-rectangular-package)
(install-polar-package)

(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))
(define make-from-real-imag (get 'make-from-real-imag 'rectangular))
(define make-from-mag-ang (get 'make-from-mag-ang 'polar))

パッケージに依存しない複素数操作

In [7]:
(define (add-complex z1 z2)
  (make-from-real-imag (+ (real-part z1) (real-part z2))
                       (+ (imag-part z1) (imag-part z2))))
(define (sub-complex z1 z2)
  (make-from-real-imag (- (real-part z1) (real-part z2))
                       (- (imag-part z1) (imag-part z2))))
(define (mul-complex z1 z2)
  (make-from-mag-ang (* (magnitude z1) (magnitude z2))
                       (+ (angle z1) (angle z2))))
(define (div-complex z1 z2)
  (make-from-mag-ang (/ (magnitude z1) (magnitude z2))
                       (- (angle z1) (angle z2))))
(define (to-list-real-imag z) (list (real-part z) (imag-part z)))
(define (to-list-mag-ang z) (list (magnitude z) (angle z)))

動作テスト

In [8]:
(add-complex (make-from-real-imag 1 1)
             (make-from-mag-ang 1 (/ pi 2)))


Traceback (most recent call last):
  File "In [8]", line 1, col 1, in 'add-complex'
  File "In [7]", line 2, col 42, in 'real-part'
  File "In [6]", line 4, col 23, in 'apply-generic'
  File "In [3]", line 11, col 3, in 'let*'
  Source "macro-generated-exp"
  File "In [3]", line 14, col 9, in 'apply'
  File "In [5]", line 6, col 43
RunTimeError: unbound variable 'cos'



## Ex 2.73

In [9]:
(define (variable? v) (symbol? v))
(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))
(define (=number? n1 n2)
  (and (number? n1) (number? n2) (= n1 n2)))

(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
        (else ((get 'deriv (operator exp))
               (operands exp) var))))
(define (operator exp) (car exp))
(define (operands exp) (cdr exp))

### a.
複合式のもつ`(operator operands...)`という構造を、
タグ付きデータの`(type-tag contents...)`の構造に見立てて微分手続きを振り分けている。

単純式（定数、変数）の場合は上記のパターンに当てはまらないので、別に結果を定義する必要がある。

### b.

In [10]:
(define (install-basic-deriviation)
  ;;internal procedure
  (define (make-sum a1 a2)
    (cond ((=number? a1 0) a2)
          ((=number? a2 0) a1)
          ((and (number? a1) (number? a2)) (+ a1 a2))
          (else (list '+ a1 a2))))
  (define (augend operands) (car operands))
  (define (addend operands) (cadr operands))
  (define (deriv-sum operands var)
    (make-sum (deriv (augend operands) var) (deriv (addend operands) var)))
  (define (make-prod m1 m2)
    (cond ((or (=number? m1 0) (=number? m2 0)) 0)
          ((=number? m1 1) m2)
          ((=number? m2 1) m1)
          ((and (number? m1) (number? m2)) (* m1 m2))
          (else (list '* m1 m2))))
  (define (multiplier operands) (car operands))
  (define (multiplicand operands) (cadr operands))
  (define (deriv-prod operands var)
    (let ((m1 (multiplier operands))
          (m2 (multiplicand operands)))
      (make-sum (make-prod (deriv m1 var) m2)
                (make-prod m1 (deriv m2 var)))))
  
  ;;interface to the rest of the system
  (put 'make '+ make-sum)
  (put 'make '* make-prod)
  (put 'deriv '+ deriv-sum)
  (put 'deriv '* deriv-prod)
  'done)

In [11]:
(install-basic-deriviation)
(deriv '(+ (* 3 (* x y)) x) 'x)

(+ (* 3 y) 1)

### c.
冪の微分なので、積に依存する。

In [12]:
(define (install-exponent-deriviation)
  ;;internal procedure
  (define make-prod (get 'make '*))
  (define (exp b e)
    (if (= e 0) 1) (* b (exp b (- e 1))))
  (define (make-exponent b e)
    (cond ((= e 1) b)
          ((or (= e 0) (=number? b 1)) 1)
          ((number? b) (exp b e))
          (else (list '** b e))))
  (define (base operands) (car operands))
  (define (exponentiation operands) (cadr operands))
  (define (deriv-exponent operands var)
    (let ((b (base operands))
          (e (exponentiation operands)))
      (make-prod e
                 (make-prod (make-exponent b (- e 1))
                            (deriv b var)))))
  
  ;;interface to the rest of the system
  (put 'make '** make-exponent)
  (put 'deriv '** deriv-exponent)
  'done)

In [13]:
(install-exponent-deriviation)
(deriv '(** x 2) 'x)

(* 2 x)

### d.
`get`と`put`の引数の順序を変えるだけ。

## Ex 2.74

大阪支社での人事データの管理

In [14]:
; record: (name address salary)
(define record1 (list 'Izumi '590-0078 190000))
(define record2 (list 'Sakai '590-0078 230000))
(define record3 (list 'Umeda '530-0001 320000))

(define (get-name-osaka rec) (car rec))
(define (get-address-osaka rec) (cadr rec))
(define (get-salary-osaka rec) (caddr rec))

; pasonnel file: (rec1 rec2 ...)
(define personnel-file-osaka (list record1 record2 record3))

(define (insert-record-osaka rec file) (cons rec file))
(define (get-record-osaka name file)
  (cond ((null? file) #f)
        ((eq? name (get-name-osaka (car file))) (car file))
        (else (get-record-osaka name (cdr file)))))

In [15]:
(get-record-osaka 'Sakai personnel-file-osaka)

(Sakai 590-0078 230000)

神戸支社での人事データ管理

In [16]:
; record: (('name . name) ('age age) ('salary . salary) ('post post))
(define record4 (list (cons 'name 'Ashiya) (cons 'age 27) (cons 'salary 210000) (cons 'post 'staff)))
(define record5 (list (cons 'name 'Miki) (cons'age 22) (cons 'salary 180000) (cons 'post 'staff)))
(define record6 (list (cons 'name 'Sannomiya) (cons 'age 31) (cons 'salary 250000) (cons 'post 'manager)))

(define (property-selector prop)
  (define (get-prop rec)
    (cond ((null? rec) (error "Bad request!" (list prop rec)))
          ((eq? prop (caar rec)) (cdar rec))
          (else (get-prop (cdr rec)))))
  get-prop)
(define get-name-kobe (property-selector 'name))
(define get-age-kobe (property-selector 'age))
(define get-salary-kobe (property-selector 'salary))
(define get-post-kobe (property-selector 'post))

; pasonnel file: ((key1 . rec1) (key2 . rec2) ...)
(define personnel-file-kobe (list (cons 'Ashiya record4)
                      (cons 'Miki record5)
                      (cons 'Sannomiya record6)))

(define (insert-record-kobe rec file)
  (cons (cons (get-name-kobe rec) rec)
        file))
(define (get-record-kobe name file)
  (cond ((null? file) #f)
        ((eq? name (caar file)) (cdar file))
        (else (get-record-kobe name (cdr file)))))

In [17]:
(get-age-kobe (get-record-kobe 'Sannomiya personnel-file-kobe))

31

### a.
この場合、操作の単位は**人事ファイル**。
ファイル全体にタグがついてさえいればよい。
`get_record`手続きをジェネリック化し、タグに応じて二つのローカルな手続きを振り分ける。

なお、`get_record`はタグ付きデータ*だけ*を取るわけではないので、
`apply-generic`手続きを用いず、直接定義した。

大阪

In [18]:
(define (install-osaka-a)
  ; record: (name address salary)
  (define record1 (list 'Izumi '590-0078 190000))
  (define record2 (list 'Sakai '590-0078 230000))
  (define record3 (list 'Umeda '530-0001 320000))
  (define (get-name rec) (car rec))
  ; pasonnel file: (rec1 rec2 ...)
  (define personnel-file (list record1 record2 record3))
  (define (get-record name file)
    (cond ((null? file) #f)
          ((eq? name (get-name (car file))) (car file))
          (else (get-record name (cdr file)))))
  
  ;interface
  (put 'file 'osaka (attach-tag 'osaka personnel-file))
  (put 'get-record-a 'osaka get-record)
  'done)

神戸

In [19]:
(define (install-kobe-a)
  ; record: (('name . name) ('age age) ('salary . salary) ('post post))
  (define record4 (list (cons 'name 'Ashiya) (cons 'age 27) (cons 'salary 210000) (cons 'post 'staff)))
  (define record5 (list (cons 'name 'Miki) (cons'age 22) (cons 'salary 180000) (cons 'post 'staff)))
  (define record6 (list (cons 'name 'Sannomiya) (cons 'age 31) (cons 'salary 250000) (cons 'post 'manager)))
  ; pasonnel file: ((key1 . rec1) (key2 . rec2) ...)
  (define personnel-file (list (cons 'Ashiya record4)
                        (cons 'Miki record5)
                        (cons 'Sannomiya record6)))
  (define (get-record name file)
    (cond ((null? file) #f)
          ((eq? name (caar file)) (cdar file))
          (else (get-record name (cdr file)))))
  
  (put 'file 'kobe (attach-tag 'kobe personnel-file))
  (put 'get-record-a 'kobe get-record)
  'done)

本社

In [20]:
(install-osaka-a)
(install-kobe-a)
(define osaka-file (get 'file 'osaka))
(define kobe-file (get 'file 'kobe))
(define (get-record-a name tagged-file)
  ((get 'get-record-a (type-tag tagged-file))
   name
   (contents tagged-file)))

(show-lines
 osaka-file
 kobe-file
 ""
 (get-record-a 'Sakai osaka-file)
 (get-record-a 'Miki kobe-file))

(osaka (Izumi 590-0078 190000) (Sakai 590-0078 230000) (Umeda 530-0001 320000))
(kobe (Ashiya (name . Ashiya) (age . 27) (salary . 210000) (post . staff)) (Miki (name . Miki) (age . 22) (salary . 180000) (post . staff)) (Sannomiya (name . Sannomiya) (age . 31) (salary . 250000) (post . manager)))

(Sakai 590-0078 230000)
((name . Miki) (age . 22) (salary . 180000) (post . staff))


shown

### b.
この場合、操作の対象は「従業員レコード」。
したがって、レコードに対して部署タグを取り付ける必要がある。
タグ付けのタイミングは、たとえば`get-record`の返値など。
（`get-record`が従業員レコードへの唯一のインターフェースである場合。）

大阪

In [21]:
(define (install-osaka-b)
  (define (get-name rec) (car rec))
  (define (get-salary rec) (caddr rec))
  (define (get-record name file)
    (cond ((null? file) #f)
          ((eq? name (get-name (car file))) (attach-tag 'osaka (car file))) ;changed
          (else (get-record name (cdr file)))))
  
  ;interface
  (put 'get-record-b 'osaka get-record)
  (put 'get-salary '(osaka) get-salary)
  'done)

神戸

In [22]:
(define (install-kobe-b)
  (define (property-selector prop)
  (define (get-prop rec)
    (cond ((null? rec) (error "Bad request!" (list prop rec)))
          ((eq? prop (caar rec)) (cdar rec))
          (else (get-prop (cdr rec)))))
  get-prop)
  (define get-salary (property-selector 'salary))
  (define (get-record name file)
    (cond ((null? file) #f)
          ((eq? name (caar file)) (attach-tag 'kobe (cdar file))) ;changed
          (else (get-record name (cdr file)))))
  
  (put 'get-record-b 'kobe get-record)
  (put 'get-salary '(kobe) get-salary)
  'done)

本社

In [28]:
(install-osaka-b)
(install-kobe-b)
(define (get-salary rec) (apply-generic 'get-salary rec))
(define (get-record-b name tagged-file)
  ((get 'get-record-b (type-tag tagged-file))
   name
   (contents tagged-file)))

(let ((rec1 (get-record-b 'Sakai osaka-file))
      (rec2 (get-record-b 'Miki kobe-file)))
  (show-lines
   rec1
   rec2
   (get-salary rec1)
   (get-salary rec2)))

(osaka Sakai 590-0078 230000)
(kobe (name . Miki) (age . 22) (salary . 180000) (post . staff))
230000
180000


shown

### c.

In [30]:
(define all-files (list osaka-file kobe-file))
(define (find-employee-record name files)
  (if (null? files) #f
      (let ((result (get-record-b name (car files))))
        (if result result
            (find-employee-record name (cdr files))))))

(show-lines
 all-files
 ""
 (find-employee-record 'Sakai all-files)
 (find-employee-record 'Miki all-files))

((osaka (Izumi 590-0078 190000) (Sakai 590-0078 230000) (Umeda 530-0001 320000)) (kobe (Ashiya (name . Ashiya) (age . 27) (salary . 210000) (post . staff)) (Miki (name . Miki) (age . 22) (salary . 180000) (post . staff)) (Sannomiya (name . Sannomiya) (age . 31) (salary . 250000) (post . manager))))

(osaka Sakai 590-0078 230000)
(kobe (name . Miki) (age . 22) (salary . 180000) (post . staff))


shown

### d.
本社および他の支社のシステムは基本的に何の変更も要らない。
新しい部署の人事システムをパッケージ化し、本社でそれを`install`すればいい。

### 付記

完成したシステムの全貌

大阪

In [44]:
(define (install-osaka)
  ; record: (name address salary)
  (define record1 (list 'Izumi '590-0078 190000))
  (define record2 (list 'Sakai '590-0078 230000))
  (define record3 (list 'Umeda '530-0001 320000))
  (define (get-name rec) (car rec))
  (define (get-address rec) (cadr rec))
  (define (get-salary rec) (caddr rec))
  ; pasonnel file: (rec1 rec2 ...)
  (define personnel-file (list record1 record2 record3))
  (define (empty? file) (null? file))
  (define (insert-record rec file) (cons rec file))
  (define (get-record name file)
    (cond ((null? file) #f)
          ((eq? name (get-name (car file))) (attach-tag 'osaka (car file)))
          (else (get-record name (cdr file)))))
    (attach-tag 'osaka (iter name file)))
  
  ;interface
  (put 'file 'osaka (attach-tag 'osaka personnel-file))
  (put 'get-name '(osaka) get-name)
  (put 'get-address '(osaka) get-address)
  (put 'get-salary '(osaka) get-salary)
  (put 'empty? '(osaka) empty?)
  (put 'insert-record 'osaka insert-record)
  (put 'get-record 'osaka get-record)
  'done)

神戸

In [42]:
(define (install-kobe)
  ; record: (('name . name) ('age age) ('salary . salary) ('post post))
  (define record4 (list (cons 'name 'Ashiya) (cons 'age 27) (cons 'salary 210000) (cons 'post 'staff)))
  (define record5 (list (cons 'name 'Miki) (cons'age 22) (cons 'salary 180000) (cons 'post 'staff)))
  (define record6 (list (cons 'name 'Sannomiya) (cons 'age 31) (cons 'salary 250000) (cons 'post 'manager)))
  (define (property-selector prop)
  (define (get-prop rec)
    (cond ((null? rec) (error "Bad request!" (list prop rec)))
          ((eq? prop (caar rec)) (cdar rec))
          (else (get-prop (cdr rec)))))
  get-prop)
  (define get-name (property-selector 'name))
  (define get-age (property-selector 'age))
  (define get-salary (property-selector 'salary))
  (define get-post (property-selector 'post))
  ; pasonnel file: ((key1 . rec1) (key2 . rec2) ...)
  (define personnel-file (list (cons 'Ashiya record4)
                        (cons 'Miki record5)
                        (cons 'Sannomiya record6)))
  (define (empty? file) (null? file))
  (define (insert-record rec file)
    (cons (cons (get-name rec) rec)
          file))
  (define (get-record name file)
    (cond ((null? file) #f)
          ((eq? name (caar file)) (attach-tag 'kobe (cdar file))) ;changed
          (else (get-record name (cdr file)))))
  ; interface
  (put 'file 'kobe (attach-tag 'kobe personnel-file))
  (put 'get-name '(kobe) get-name)
  (put 'get-age '(kobe) get-age)
  (put 'get-salary '(kobe) get-salary)
  (put 'get-post '(kobe) get-post)
  (put 'empty? '(osaka) empty?)
  (put 'insert-record 'kobe insert-record)
  (put 'get-record 'kobe get-record)
  'done)

本社

In [39]:
(install-osaka)
(install-kobe)
(define osaka-file (get 'file 'osaka))
(define kobe-file (get 'file 'kobe))
(define all-files (list osaka-file kobe-file))
(define (get-name rec) (apply-generic 'name rec))
(define (get-salary rec) (apply-generic 'get-salary rec))
(define (insert-record rec tagged-file)
  ((get 'insert-record (type-tag tagged-file))
   rec
   (contents tagged-file)))
(define (get-record name tagged-file)
  ((get 'get-record (type-tag tagged-file))
   name
   (contents tagged-file)))
(define (find-employee-record name files)
  (if (null? files) #f
      (let ((result (get-record-b name (car files))))
        (if result result
            (find-employee-record name (cdr files))))))

### Ex 2.75
説明不要。
データが手続きを持っているという意味で、
オブジェクト指向にかなり近い。

In [46]:
(define (make-from-mag-ang r a)
  (define (dispatch op)
    (cond ((eq? op 'real-part) (* r (cos a)))
          ((eq? op 'imag-part) (* r (sin a)))
          ((eq? op 'magnitude) r)
          ((eq? op 'angle) a)
          (else (error "Unknown op: MAKE-FROM-MAG-ANG" op))))
  dispatch)

## Ex 2.76
1. explicit generic procedure
```
(define (op1 tagged-data)
    (cond ((eq? (type-tag tagged-data) 'type1) (op1-for-type1 (contents tagged-data)))
    ...))
```
2. data directed programming
```
(put 'op1 'type1 op1-for-type1)...
(define (op1 tagged-data)
    ((get 'op1 (type-tag tagged-data)) (contents tagged-data)))...
```
3. message passing
```
(define (make-type1 ...)
    (define (dispatch op)
        (cond ((eq? op 'op1) result-of-op1-for-type1)
        ...)
    dispatch)
```

- タイプを追加する場合
    1. 既存のジェネリック手続きをすべて変更する必要がある。
    2. 新たなパッケージを定義し、既存の手続きを実装。それをインストールすればよい。（ジェネリック手続き自体は変更不要）
    3. 新たなコンストラクタを定義し、内部で既存の手続きに対して実装を与えればよい。
- 手続きを追加する場合
    1. 新たなジェネリック手続きを定義する。
    2. 既存のパッケージをすべて変更する必要がある。
    3. 既存のコンストラクタをすべて変更する必要がある。

「変更がどこまで波及するか」という観点で見るなら、
* タイプを追加する場合は、2.や3.が有利。
* 手続きを追加する場合は、1.が有利。